In [1]:
# Import Splinter, BeautifulSoup, and Pandas

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep, ctime



In [2]:
# Path to chromedriver

path = "/usr/local/bin/chromedriver"


In [3]:
# Set the executable path and initialize the chrome browser in splinter

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_mac64.zip
Driver has been saved in cache [/Users/wolfey/.wdm/drivers/chromedriver/mac64/94.0.4606.61]


## Visit the NASA mars news site

In [4]:
# Visit the mars nasa news site
nasa_url = "https://mars.nasa.gov/news/"

browser.visit(nasa_url)

# Optional delay for loading the page
sleep(5)


In [5]:
# Convert the browser html to a soup object and then quit the browser

html = browser.html
soup = bs(html, 'html.parser')



In [6]:
# .find() the content title and save it as `news_title`

article = soup.find_all('div', class_='content_title')
news_title = article[1].text
news_title


"You Can Help Train NASA's Rovers to Better Explore Mars"

In [7]:
# .find() the paragraph text

paragraph_text = soup.find('div', class_='article_teaser_body').text
paragraph_text


'Members of the public can now help teach an artificial intelligence algorithm to recognize scientific features in images taken by NASA’s Perseverance rover.'

## JPL Space Images Featured Image

In [8]:
# Visit JPL space images Mars URL 

jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(jpl_url)

In [9]:
# Find and click the full image button

browser.find_by_xpath('/html/body/div[1]/div/a/button').click()


In [10]:
# Find the more info button and click that


In [11]:
# Parse the resulting html with soup

html = browser.html
soup =  bs(html, 'html.parser')


In [12]:
# find the relative image url

relative_image_url = soup.find('img', class_="fancybox-image")["src"]
relative_image_url


'image/featured/mars3.jpg'

In [13]:
# Use the base url to create an absolute url
absolute_image_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{relative_image_url}'
absolute_image_url


'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

## Mars Facts

In [14]:
# Create a dataframe from the space-facts.com mars page
mars = 'https://space-facts.com/mars/'
mars_df = pd.read_html(mars)
mars_df = mars_df[1]

mars_df


,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# clean the dataframe and export to HTML

mars_df.to_html('table.html')


## Hemispheres

In [ ]:
# visit the USGS astrogeology page for hemisphere data from Mars

In [19]:


# First, get a list of all of the hemispheres

# Next, loop through those links, click the link, find the sample anchor, return the href
    
    # We have to find the elements on each loop to avoid a stale element exception
    
    
    # Next, we find the Sample image anchor tag and extract the href
    
    
    # Get Hemisphere title
    
    
    # Append hemisphere object to list
    
    
    # Finally, we navigate backwards with browser.back()
    

In [20]:
# view the hemisphere urls to make sure they look good

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]